In [56]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification
from sklearn import datasets
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

import scipy.stats
import nltk
import string
import sklearn.metrics
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import os
import csv
import matplotlib.pyplot as plt
import nltk

In [57]:
df = pd.read_csv('tokped.csv',encoding='utf-8',error_bad_lines=False)
df.Status.replace(['Negativ','Positiv'],[0,1],inplace=True)

In [58]:
df.head(10)

,No,Komentar,Status
0,1,dapet mas barang pagi malam dasar anak,0
1,2,flash sale akalan rajin mantengin jam pagi mal...,0
2,3,lihat review orang orang pas beli hp harga fla...,1
3,4,nyobain cuma iseng pembayaran mau jam langsung...,1
4,5,pemberi harapan palsu doang kena tipu kayak ke...,0
5,6,bukalapak wkwkw tokopedia ketinggalan ulang ta...,1
6,7,dapet namanya juga flash sale bawa santai dape...,1
7,8,stock tersedia pas bayar tulisan stock habis k...,0
8,9,ga pernah kebagian astagaaahhh,1
9,10,beneran lihat kepala mata walaubsaya mengakui ...,1


In [59]:
df.columns

Index(['No', 'Komentar', 'Status'], dtype='object')

In [60]:
print(df.shape)

(1225, 3)


In [61]:
print(df.iloc[[10, -4]])

        No Komentar  Status
10      11   boicot       0
1221  1222     pagi       0


In [62]:
#menampilkan jumlah missing data pada setiap kolom
df.isnull().sum()

No          0
Komentar    0
Status      0
dtype: int64

In [63]:
#membuang setiap baris yang didalamnya terdapat missing data
df_clean = df.dropna()
df_clean.describe()

,No,Status
count,1225.000000,1225.000000
mean,613.000000,0.493878
std,353.771348,0.500167
min,1.000000,0.000000
25%,307.000000,0.000000
50%,613.000000,0.000000
75%,919.000000,1.000000
max,1225.000000,1.000000


In [64]:
num_of_numerical_cols = df_clean._get_numeric_data().columns.shape[0]
print(num_of_numerical_cols, 'numerical columns')
name_of_numerical_cols = df_clean._get_numeric_data().columns
print(name_of_numerical_cols)

2 numerical columns
Index(['No', 'Status'], dtype='object')


In [65]:
words = df["Komentar"].str.split()
df.iloc[7]

No                                                          8
Komentar    stock tersedia pas bayar tulisan stock habis k...
Status                                                      0
Name: 7, dtype: object

In [66]:
import nltk
#nltk.download()
from nltk.corpus import stopwords
stop = stopwords.words('Indonesian')
df["Komentar"] = df["Komentar"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df["Komentar"] = df["Komentar"].apply(lambda x: x.lower())
all_words = df["Komentar"].str.split()
all_words = nltk.FreqDist(df["Komentar"])
print(all_words.most_common(15))
#df.head(5)

[('amazing', 10), ('dapet', 10), ('transaksi flash sale kemarin pembayaran via bca virtual account dana terpotong sistem error belanjaan stuck cart produk habis hilang skrg dana terpotong cart logo notification dibuka cart kosong mohon kebijakan tokopedia bagaimanakesal', 10), ('ok', 10), ('gagal beli uang toko bayar langsung ditarik saldo trs dibalikin bank dasar anak informasi', 10), ('flash sale akalan rajin mantengin jam pagi malem sampe nyalain alarm tetep zonk server stock habis coba ku kasar wkwkwkekwk', 5), ('pemberi harapan palsu doang kena tipu kayak keledai', 5), ('beneran lihat kepala mata walaubsaya mengakui kalo sedih bete kena marah', 5), ('boicot', 5), ('teflon frame', 5), ('koneksi indomie calon anak ayam kornet stabil bagus', 5), ('lom beruntung kemarin marah hikz', 5), ('norak kali ikutan flash salekesal wkwk', 5), ('barang', 5), ('nature republic', 5)]


In [67]:
import nltk
words = set(nltk.corpus.words.words())
df["Komentar"] = df["Komentar"].apply(lambda x: ' '.join([word for word in x.split() if word in (words)]))
df.head(5)

,No,Komentar,Status
0,1,mas,0
1,2,flash sale jam alarm server stock,0
2,3,review orang orang flash sale,1
3,4,jam stock bin,1
4,5,kayak,0


In [68]:
# Data training
X = df[['Komentar']]
Y = df[['Status']]
x_train, x_test, y_train, y_test = train_test_split(X['Komentar'],Y['Status'],test_size=0.2,random_state=0) # 70% training and 30% test

In [69]:
print('x_train=',x_train.shape)
print('x_test=',x_test.shape)
print('y_train=',y_train.shape)
print('y_test=',y_test.shape)

x_train= (980,)
x_test= (245,)
y_train= (980,)
y_test= (245,)


In [70]:
vec = CountVectorizer(max_features=10000,binary=True)
x_train_vect = vec.fit_transform(x_train)
counts = df.Status.value_counts()
print(counts) # 0 = NEGATIF, 1 = POSITIF

0    620
1    605
Name: Status, dtype: int64


In [71]:
vec = CountVectorizer()
vec.fit(X['Komentar'])

x_train = vec.transform(x_train)

x_test = vec.transform(x_test)

In [72]:
NB = MultinomialNB(alpha=.3)
#print(y_train)
NB.fit(x_train,y_train)
NB_predicted = NB.predict(x_test)
NBC = NB.score(x_test,y_test)
print("\nAccuracy of Naive Bayes Classifier: ",NBC*100,"%\n")


Accuracy of Naive Bayes Classifier:  71.42857142857143 %



In [73]:
S = LinearSVC()
S.fit(x_train,y_train)
S_predicted = S.predict(x_test)
SC = S.score(x_test,y_test)
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))
print("\nAccuracy of Linear Support Vector Classifier: ",SC*100,"%\n")

[[42 86]
 [39 78]]
              precision    recall  f1-score   support

           0       0.52      0.33      0.40       128
           1       0.48      0.67      0.56       117

   micro avg       0.49      0.49      0.49       245
   macro avg       0.50      0.50      0.48       245
weighted avg       0.50      0.49      0.48       245


Accuracy of Linear Support Vector Classifier:  72.6530612244898 %



In [74]:
x_test_vect = x_test
y_pred = NB_predicted

In [75]:
confussion_array = sklearn.metrics.confusion_matrix(y_test, NB_predicted)
print(confussion_array)

[[109  19]
 [ 51  66]]


In [76]:
confussion_array = sklearn.metrics.confusion_matrix(y_test, S_predicted)
print(confussion_array)

[[ 72  56]
 [ 11 106]]


In [77]:
print('TN = ', confussion_array[0,0]) #True Negative
print('FN = ', confussion_array[1,0]) #False Negative
print('TP = ', confussion_array[1,1]) #True Negative
print('FP = ', confussion_array[0,1]) #False Negative

TN =  72
FN =  11
TP =  106
FP =  56


In [78]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

print("Accuracy : {:.2f}%".format(accuracy_score(y_test,y_pred)*100))
print("F1 Score : {:.2f}%".format(f1_score(y_test,y_pred)*100))
print("Confusion Matrix : \n",confusion_matrix(y_test,y_pred))

Accuracy : 71.43%
F1 Score : 65.35%
Confusion Matrix : 
 [[109  19]
 [ 51  66]]


In [84]:
# Hasil Akurasi, F1 Measure, Confusion Matrix dengan Cross-validation
from sklearn.model_selection import ShuffleSplit

X = df.Komentar
y = df.Status

ss = ShuffleSplit(n_splits=10, test_size=0.2)

accs = []
fls = []
cms = []
hsl = 0
hsl1 = 0
hsl2 = 0
#Proses K-Folds cross-validation
for train_index, test_index, in ss.split(X):
    x_train, x_test = X.iloc[train_index],X.iloc[test_index]
    y_train, y_test = y.iloc[train_index],y.iloc[test_index]
    
    X_train_vect = vec.fit_transform(x_train)
    X_test_vect = vec.transform(x_test)
    
    #X_train_res, y_train_res = sm.fit_sample(X_train_vect,y_train)
    
    #NB.fit(X_train_res,y_train_res)
    y_pred = S.predict(X_test_vect)
    
    accs.append(accuracy_score(y_test, y_pred))
    fls.append(f1_score(y_test, y_pred))
    cms.append(confusion_matrix(y_test, y_pred))   
    
    hsl += accuracy_score(y_test, y_pred)
    hsl1 += f1_score(y_test, y_pred)
    hsl2 += confusion_matrix(y_test, y_pred)
    
    
print("\nAverage accuracy across folds: {:.2f}%".format(hsl/len(accs)*100))
print("\nAverage F1 score across folds: {:.2f}%".format(hsl1/len(fls) * 100))
print("\nAverage Confusion Matrix across folds: \n {}".format(hsl2 / len(cms)))


Average accuracy across folds: 73.84%

Average F1 score across folds: 77.64%

Average Confusion Matrix across folds: 
 [[ 69.3  56.4]
 [  7.7 111.6]]
